In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
from city_sustainability.models import unet_model

In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    Dropout,
    MaxPooling2D,
    UpSampling2D,
    Concatenate,
    Input,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16


def encoder(inputs):
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.1)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    return conv1, conv2, pool2


def decoder(conv1, conv2, encoded):
    up1 = UpSampling2D(size=(2, 2))(encoded)
    merge1 = Concatenate(axis=3)([conv2, up1])
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge1)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.1)(conv3)

    up2 = UpSampling2D(size=(2, 2))(conv3)
    merge2 = Concatenate(axis=3)([conv1, up2])
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge2)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.1)(conv4)

    return conv4


def build_model(input_shape=(28, 28, 1), num_classes=12):
    inputs = Input(input_shape)
    conv1, conv2, encoded = encoder(inputs)
    decoded = decoder(conv1, conv2, encoded)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(decoded)
    return Model(inputs=inputs, outputs=outputs)



def build_vgg16_model(input_shape=(28, 28, 1), num_classes=12 ):
    model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    for layer in model_vgg.layers:
        layer.trainable = False
    inputs = model_vgg.layers[0].output
    conv1 = model_vgg.get_layer('block1_conv2').output
    conv2 = model_vgg.get_layer('block2_conv2').output
    encoded = model_vgg.get_layer('block3_conv3').output
    decoded = decoder(conv1, conv2, encoded)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(decoded)
    return Model(inputs=inputs, outputs=outputs)




def compile_model(model, optimizer='adam'):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', compute_iou])
def train_model(model, train_generator, epochs=1, steps_per_epoch=None, validation_data=None, validation_steps=None):
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001, verbose=1)
    early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
              validation_data=validation_data, validation_steps=validation_steps,
              callbacks=[lr_reducer, early_stopper])


def compute_iou(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) - intersection
    iou = tf.reduce_mean((intersection + 1e-7) / (union + 1e-7))
    return iou


# def train_model(model, train_generator, epochs=1, batch_size=32, validation_split=0.1):
#     model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', compute_iou])
#     lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001, verbose=1)
#     early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
#     model.fit(train_generator, epochs=epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[lr_reducer, early_stopper])



def evaluate_model(model, x, y):
    loss, accuracy = model.evaluate(x, y)
    
    # Calculate IoU
    y_pred = model.predict(x)
    iou = compute_iou(y, y_pred)
    
    print("Evaluation results:")
    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"IoU: {iou:.4f}")



def predict(model, x):
    return model.predict(x)


In [39]:
import os
import numpy as np
import imageio
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Set the directory paths for images and labels
image_directory = '/home/mgudipati/code/Zubairslb/city_sustainability/raw_data/resize_train_all/images'
label_directory = '/home/mgudipati/code/Zubairslb/city_sustainability/raw_data/resize_train_all/labels'

# Set the target image size
target_size = (256, 256)

# Set the batch size and number of classes
batch_size = 16
num_classes = 9

# Create an ImageDataGenerator for image augmentation and scaling
image_datagen = ImageDataGenerator(
    rescale=1.0/255.0  # Scale pixel values between 0 and 1
)

# Create the image generator
image_generator = image_datagen.flow_from_directory(
    image_directory,
    target_size=target_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,  # Do not load class labels initially
    seed=42,  # Set a seed for reproducibility
    shuffle=True  # Shuffle the order of the images
)

# Create the label generator
label_generator = image_datagen.flow_from_directory(
    label_directory,
    target_size=target_size,
    color_mode='grayscale',  # Load label images in grayscale
    batch_size=batch_size,
    class_mode=None,  # Do not load class labels initially
    seed=42,  # Set a seed for reproducibility
    shuffle=True  # Shuffle the order of the images
)

# Combine the image and label generators using zip
train_generator = zip(image_generator, label_generator)

# Convert labels to one-hot encoding using to_categorical
def preprocess_labels(labels):
    categorical_labels = to_categorical(labels, num_classes=num_classes)
    return categorical_labels

# # Apply one-hot encoding to the labels
train_generator = ((images, preprocess_labels(labels)) for images, labels in train_generator)
# train_generato__r = ((images, labels) for images, labels in train_generator)


# Load a sample TIFF image for testing
# sample_tiff_path = '../resize_train_all/images/aachen_1.tif'
# sample_tiff_image = imageio.imread(sample_tiff_path)
# sample_tiff_array = np.array(sample_tiff_image)

# Print the shape and data type of the loaded TIFF image
# print("Sample TIFF image shape:", sample_tiff_array.shape)
# print("Sample TIFF image data type:", sample_tiff_array.dtype)


Found 2303 images belonging to 1 classes.
Found 2303 images belonging to 1 classes.


In [40]:
for x in train_generator:
    print(x[0][1])
    break

[[[0.43137258 0.35686275 0.3647059 ]
  [0.4156863  0.3372549  0.34117648]
  [0.41960788 0.3254902  0.31764707]
  ...
  [0.64705884 0.6392157  0.6509804 ]
  [0.7372549  0.72156864 0.7176471 ]
  [0.7372549  0.72156864 0.7254902 ]]

 [[0.56078434 0.47450984 0.48235297]
  [0.45882356 0.3372549  0.31764707]
  [0.42352945 0.32941177 0.32941177]
  ...
  [0.5176471  0.4666667  0.43529415]
  [0.60784316 0.60784316 0.6156863 ]
  [0.7254902  0.70980394 0.7058824 ]]

 [[0.43137258 0.32941177 0.33333334]
  [0.4431373  0.29411766 0.25882354]
  [0.43137258 0.33333334 0.34901962]
  ...
  [0.57254905 0.4039216  0.2784314 ]
  [0.7607844  0.72156864 0.7137255 ]
  [0.70980394 0.7019608  0.7058824 ]]

 ...

 [[0.65882355 0.6156863  0.5921569 ]
  [0.5921569  0.56078434 0.54901963]
  [0.63529414 0.6156863  0.6039216 ]
  ...
  [0.6392157  0.48627454 0.45882356]
  [0.61960787 0.5411765  0.5137255 ]
  [0.7411765  0.74509805 0.72156864]]

 [[0.65882355 0.6156863  0.5921569 ]
  [0.6392157  0.60784316 0.59607846]


In [41]:
# Create the model
model = build_vgg16_model(input_shape=(256, 256, 3), num_classes=9)

# Compile the model
compile_model(model)

# Train the model
train_model(model, train_generator, epochs=1)


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'generator'>]

In [43]:
image_directory = '/home/mgudipati/code/Zubairslb/city_sustainability/raw_data/resize_train_all/images'
label_directory = '/home/mgudipati/code/Zubairslb/city_sustainability/raw_data/resize_train_all/labels'

image_datagen = ImageDataGenerator(validation_split=0.2)
mask_datagen = ImageDataGenerator(validation_split=0.2)

seed = 123

image_generator = image_datagen.flow_from_directory(
    image_directory,
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(256, 256),
    subset = 'training',
    shuffle=False)

mask_generator = mask_datagen.flow_from_directory(
    label_directory,
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(256, 256),
    subset = 'training',
    shuffle=False
   )

val_image_generator = image_datagen.flow_from_directory(
    image_directory,
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(256, 256),
    subset = 'validation',
    shuffle=False)

val_mask_generator = mask_datagen.flow_from_directory(
    label_directory,
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(256, 256),
    subset = 'validation',
    shuffle=False
   )


train_generator = zip(image_generator, mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

def preprocess_labels(labels):
    categorical_labels = to_categorical(labels, num_classes=num_classes)
    return categorical_labels

train_generator = ((images, preprocess_labels(labels)) for images, labels in train_generator)
val_generator = ((images, preprocess_labels(labels)) for images, labels in val_generator)



Found 1843 images belonging to 1 classes.
Found 1843 images belonging to 1 classes.
Found 460 images belonging to 1 classes.
Found 460 images belonging to 1 classes.


In [50]:

# model = get_model(img_hieght, img_width, img_channel, num_classes)

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

# history = model.fit(train_generator,
#                     verbose=1,
#                     epochs=100,
#                     steps_per_epoch= image_generator.samples//batch_size,
#                     validation_data= val_generator,
#                     validation_steps= val_image_generator.samples//batch_size
#                     )

# Create the model
model = build_vgg16_model(input_shape=(256, 256, 3), num_classes=9)

# Compile the model
compile_model(model)

# Train the model
train_model(model, train_generator,
                    epochs=10,
                    steps_per_epoch= image_generator.samples//batch_size,
                    validation_data= val_generator,
                    validation_steps= val_image_generator.samples//batch_size
                    )

Epoch 1/10


2023-05-26 18:40:03.509664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


115/115 [==============================] - ETA: 0s - loss: 2.1224 - accuracy: 0.2640 - compute_iou: 0.1003

2023-05-26 18:50:40.797094: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


115/115 [==============================] - 692s 6s/step - loss: 2.1224 - accuracy: 0.2640 - compute_iou: 0.1003 - val_loss: 1.9304 - val_accuracy: 0.3405 - val_compute_iou: 0.1546 - lr: 1.0000e-04
Epoch 2/10
115/115 [==============================] - 674s 6s/step - loss: 1.8143 - accuracy: 0.3636 - compute_iou: 0.1346 - val_loss: 1.6035 - val_accuracy: 0.4176 - val_compute_iou: 0.1611 - lr: 1.0000e-04
Epoch 3/10
115/115 [==============================] - 677s 6s/step - loss: 1.7196 - accuracy: 0.3976 - compute_iou: 0.1514 - val_loss: 1.5402 - val_accuracy: 0.4488 - val_compute_iou: 0.1678 - lr: 1.0000e-04
Epoch 4/10
 72/115 [=================>............] - ETA: 3:55 - loss: 1.6394 - accuracy: 0.4234 - compute_iou: 0.1656